In [3]:
"""Some examples for quick testing/demonstrations.

All function accept `show` and `draw` arguments

  * If `draw` is `True` it will return the widgets (Scatter, Volume, Mesh)
  * If `draw` is `False`, it will return the data
  * if `show` is `False`, `ipv.show()` will not be called.
"""
#import sys
#sys.path.append("..")
import warnings
import numpy as np
from numpy import cos, sin, pi
#import ipyvolume.pylab as p3
from ipyvolume import pylab as p3
import math
from ipywidgets import widgets
from ipywidgets import FloatRangeSlider, Dropdown, FloatSlider, ColorPicker
from ipywidgets import interact, interactive, fixed, interact_manual

try:
    import scipy.ndimage
    import scipy.special
except:
    pass  # it's ok, it's not crucial
# __all__ = ["example_ylm"]

def klein_bottle(
    draw=True,
    show=True,
    figure8=False,
    endpoint=True,
    uv=True,
    wireframe=False,
    texture=None,
    both=False,
    interval=1000,
):
    # http://paulbourke.net/geometry/klein/
    u = np.linspace(0, 2 * pi, num=50, endpoint=endpoint)
    v = np.linspace(0, 2 * pi, num=50, endpoint=endpoint)
    u, v = np.meshgrid(u, v)
    r = 4 * (1 - cos(u) / 2)
    x = 6 * cos(u) * (1 + sin(u)) + r * cos(u) * cos(v) * (u < pi) + r * cos(v + pi) * (u >= pi)
    y = 16 * sin(u) + r * sin(u) * cos(v) * (u < pi)
    z = r * sin(v)
    mesh = p3.plot_mesh(
                x,
                y,
                z,
                wrapx=not endpoint,
                wrapy=not endpoint,
                u=u / (2 * np.pi),
                v=v / (2 * np.pi),
                wireframe=wireframe,
                texture=texture,
            )

    return mesh
def worldplane(separate_figure=False):
    k = 20
    h = -15
    tx = np.array([k, -k, -k, k])
    tz = np.array([k, k, -k, -k])
    ty = np.array([h, h, h, h])
    
    tri = [(0, 1, 2), (0, 2, 3)]
    p3.plot_trisurf(tx, ty, tz, triangles=tri, color='orange')
    
    if separate_figure:
        p3.figure()
        p3.show()
def ambient_light_widget():
    selected_size =  FloatSlider(value=5, min=0, max=10, step=1, description='Intensity:', disabled=False,
                 continuous_update=True,orientation='horizontal', readout=True, readout_format='d')
    selected_color = ColorPicker(concise=False, description='Ambient Color:', value='red', disabled=False)
    display(selected_size,selected_color)
    #widgets.interact(p3.ambient_light,light_color=selected_color,intensity=selected_size);
    print(selected_size.value)
    return selected_size

def add_lights():
    #ambient_light_widget()

    #p3.ambient_light(light_color="rgb(255,0,0)", intensity=1.5)

    p3.directional_light(light_color='cyan', intensity=1.0, position=[100, 100, 100], target=[0,0,0], cast_shadow=True, shadow_bias= -0.005,shadow_camera_orthographic_size=100)
    p3.directional_light(light_color='orange', intensity=1.0, position=[-100, 100, -100], target=[0,0,0], cast_shadow=True, shadow_bias= -0.005,shadow_camera_orthographic_size=100)
    #p3.hemisphere_light(light_color='red', color2='yellow', intensity=0.5, position=[1, 0, 0])
    #p3.point_light(light_color='orange', intensity=1.0, position=[20, 30, 20], distance=65, decay=0.5, cast_shadow=True,shadow_radius=3,shadow_bias= -0.005)
    #p3.spot_light(light_color='yellow', intensity=1, position=[20, 30, 20], angle=math.pi/9, target = [-20,-20,-20], cast_shadow=True,shadow_radius=2)



#very important to first call p3.clear()
p3.clear()
klein_bottle()
worldplane()

p3.show()
add_lights()


[Mesh(line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSide'), texture=None, triangles=array([[   0,   50,   51],
       [   0,   51,    1],
       [   1,   51,   52],
       ...,
       [2447, 2498, 2448],
       [2448, 2498, 2499],
       [2448, 2499, 2449]], dtype=uint32), u=array([0.        , 0.02040816, 0.04081633, ..., 0.95918367, 0.97959184,
       1.        ]), v=array([0., 0., 0., ..., 1., 1., 1.]), x=array([8.        , 8.71156903, 9.27378258, ..., 2.26620625, 3.17335636,
       4.        ]), y=array([ 0.00000000e+00,  2.30388865e+00,  4.58237413e+00, ...,
       -4.05847334e+00, -2.04603459e+00, -3.91886976e-15]), z=array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -5.05879616e-16, -4.93880453e-16, -4.89858720e-16])), Mesh(color=array('orange', dtype='<U6'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSide'), texture=None, triangles=array([[0, 1, 2],
       [0, 2, 3]], dtype=uint32), x=array([ 20, -20, -20,  20]),

ADD DIRECTIONAL LIGHT (from pylab) 
light_color: cyan
intensity: 1.0
position: 100 100 100
target: 0 0 0
cast_shadow: True
shadow_map_size: 512
shadow_bias: -0.005
shadow_radius: 1
shadow_camera_near: 0.5
shadow_camera_far: 500
ADD DIRECTIONAL LIGHT (from pylab) 
light_color: orange
intensity: 1.0
position: -100 100 -100
target: 0 0 0
cast_shadow: True
shadow_map_size: 512
shadow_bias: -0.005
shadow_radius: 1
shadow_camera_near: 0.5
shadow_camera_far: 500


In [12]:
def head2(draw=True, show=True, max_shape=256):
    """Show a volumetric rendering of a human male head."""
    # inspired by http://graphicsrunner.blogspot.com/2009/01/volume-rendering-102-transfer-functions.html
    import ipyvolume as ipv
    from scipy.interpolate import interp1d

    # First part is a simpler version of setting up the transfer function. Interpolation with higher order
    # splines does not work well, the original must do sth different
    colors = [[0.91, 0.7, 0.61, 0.0], [0.91, 0.7, 0.61, 80.0], [1.0, 1.0, 0.85, 82.0], [1.0, 1.0, 0.85, 256]]
    x = np.array([k[-1] for k in colors])
    rgb = np.array([k[:3] for k in colors])
    N = 256
    xnew = np.linspace(0, 256, N)
    tf_data = np.zeros((N, 4))
    kind = 'linear'
    for channel in range(3):
        f = interp1d(x, rgb[:, channel], kind=kind)
        ynew = f(xnew)
        tf_data[:, channel] = ynew
    alphas = [[0, 0], [0, 40], [0.2, 60], [0.05, 63], [0, 80], [0.9, 82], [1.0, 256]]
    x = np.array([k[1] * 1.0 for k in alphas])
    y = np.array([k[0] * 1.0 for k in alphas])
    f = interp1d(x, y, kind=kind)
    ynew = f(xnew)
    tf_data[:, 3] = ynew
    tf = ipv.TransferFunction(rgba=tf_data.astype(np.float32))

    head_data = ipv.datasets.head.fetch().data
    if draw:
        vol = ipv.volshow(head_data, tf=tf, max_shape=max_shape)
        if show:
            ipv.show()
        return vol
    else:
        return head_data
    
p3.clear()
head2()

Volume(data=array([[[ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1,…